In [1]:
#https://stackoverflow.com/questions/46160897/how-to-format-a-python-script-that-uses-rpy2-in-order-to-build-a-model-with-an-r
import pandas as pd
import rpy2.robjects as r
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector
from rpy2.robjects import r, pandas2ri

In [2]:
pandas2ri.activate()
caret = rpackages.importr('caret')
base = rpackages.importr('base')

In [3]:
# Read data make necessary change for logistic model
data = pd.read_csv("../Data/lr_version_merged_mean.csv")
data["Average_bleaching"] = ["yes" if x['Average_bleaching'] >0 else "no" for idx, x in data.iterrows()]
# data["Average_bleaching"] = [1 if x['Average_bleaching'] >0 else 0 for idx, x in data.iterrows()]
# data["Average_bleaching"] = data["Average_bleaching"].astype("category")

In [4]:
data.head()

,index,Year,Month,reef_longitude,reef_latitude,S1,S2,S3,S4,Average_bleaching,...,rate_of_SST_change,eta,salt,temp,wspeed_u,wspeed_v,mean_wspeed,u,v,mean_cur
0,5097,2010,10,153.491389,-26.968611,0.0,0.0,0.0,0.0,no,...,0.038889,-0.330480,35.194298,23.267048,-3.294642,1.381906,6.046273,-0.044380,-0.259013,0.413241
1,5102,2010,10,153.483333,-26.968333,0.0,0.0,0.0,0.0,no,...,0.038889,-0.330480,35.194298,23.267048,-3.294642,1.381906,6.046273,-0.044380,-0.259013,0.413241
2,5106,2010,11,153.385556,-27.529167,2.0,2.0,3.0,1.0,yes,...,0.041288,-0.298065,35.172879,24.257499,-3.813082,-0.412349,6.041818,0.005312,-0.349575,0.469270
3,5132,2010,9,153.151667,-26.635000,0.0,0.0,0.0,0.0,no,...,0.038889,-0.228805,35.578607,22.283628,-0.172312,-0.509025,5.732539,0.020565,-0.063561,0.229096
4,5134,2010,11,153.103056,-26.359722,0.0,0.0,0.0,0.0,no,...,0.038889,-0.287245,35.449339,23.996596,-4.567952,0.244954,6.144467,-0.028463,-0.151352,0.250832


In [5]:
# Convert pandas dataframe to r dataframe
r_dataframe = pandas2ri.py2rpy(data)

In [6]:
# Turn average_bleaching column to factor
# import rpy2
col_index = list(r_dataframe.colnames).index('Average_bleaching')
# col_factor = rpy2.robjects.vectors.FactorVector(r_dataframe.rx2('Average_bleaching'))
# r_dataframe[col_index] = col_factor

In [7]:
r_dataframe[col_index] = base.as_character(r_dataframe[col_index])

In [8]:
print(r_dataframe[col_index])

  [1] "no"  "no"  "yes" "no"  "no"  "yes" "yes" "no"  "yes" "yes" "no"  "no" 
 [13] "yes" "yes" "yes" "yes" "yes" "yes" "yes" "no"  "yes" "yes" "yes" "no" 
 [25] "yes" "yes" "yes" "yes" "yes" "no"  "yes" "no"  "yes" "yes" "yes" "no" 
 [37] "yes" "yes" "no"  "yes" "yes" "yes" "yes" "yes" "no"  "no"  "yes" "no" 
 [49] "no"  "no"  "yes" "yes" "yes" "yes" "yes" "yes" "yes" "yes" "no"  "no" 
 [61] "yes" "yes" "no"  "yes" "yes" "yes" "yes" "no"  "yes" "yes" "yes" "yes"
 [73] "no"  "no"  "yes" "no"  "no"  "yes" "yes" "yes" "yes" "yes" "yes" "no" 
 [85] "yes" "yes" "yes" "yes" "yes" "yes" "yes" "yes" "yes" "yes" "yes" "yes"
 [97] "yes" "yes" "yes" "yes" "yes" "yes" "yes" "yes" "no"  "yes" "yes" "yes"
[109] "no"  "yes" "yes" "no"  "yes" "yes" "yes" "yes" "yes" "yes" "yes" "yes"
[121] "no"  "yes" "yes" "yes" "yes" "yes" "no"  "yes" "yes" "yes" "yes" "yes"
[133] "yes" "yes" "yes" "no"  "yes" "yes" "yes" "yes" "yes" "yes" "yes" "yes"
[145] "yes" "yes" "yes" "yes" "yes" "yes" "yes" "yes" "yes" "yes

In [9]:
# Specify predictor columns
predictors = ["ClimSST", "Temperature_Kelvin", "Temperature_Kelvin_Standard_Deviation", "SSTA_Frequency", "SSTA_Frequency_Standard_Deviation" ,"TSA_Frequency_Standard_Deviation" ,"mean_cur"]
predictors_vector = StrVector(predictors)

In [10]:
# Specify outcome column
outcome = ["Average_bleaching"]
outcome_vector = StrVector(outcome)

In [11]:
# Extract columns of predictors and outcome from dataframe
columns_predictors = r_dataframe.rx(True, predictors_vector)
columns_response= r_dataframe.rx(True, outcome_vector)

In [12]:
# Train and predict
cvCtrl = caret.trainControl(method = "cv", number= 10, savePredictions=True)

model_R= caret.train(columns_predictors, columns_response, method = "glm", trControl = cvCtrl,family="binomial")

print(model_R.rx('finalModel'))

$finalModel

Call:  NULL

Coefficients:
                          (Intercept)                                ClimSST  
                            -85.70286                                0.05948  
                   Temperature_Kelvin  Temperature_Kelvin_Standard_Deviation  
                              0.21725                                1.31316  
                       SSTA_Frequency      SSTA_Frequency_Standard_Deviation  
                             -0.09264                                0.62745  
     TSA_Frequency_Standard_Deviation                               mean_cur  
                             -0.03850                                0.25282  

Degrees of Freedom: 270 Total (i.e. Null);  263 Residual
Null Deviance:	    243.7 
Residual Deviance: 211.6 	AIC: 227.6




In [13]:
print(model_R.rx("resample"))

$resample
    Accuracy       Kappa Resample
1  0.9259259  0.63013699   Fold01
2  0.8214286  0.20454545   Fold02
3  0.7407407 -0.11834320   Fold03
4  0.7857143 -0.06329114   Fold04
5  0.8076923  0.17721519   Fold05
6  0.8518519  0.00000000   Fold06
7  0.8571429  0.29113924   Fold07
8  0.8461538  0.25714286   Fold08
9  0.8518519  0.00000000   Fold09
10 0.8518519  0.28947368   Fold10




In [25]:
test_df = pd.DataFrame([[297.28, 295.6, 2.15, 12, 3.34, 1.1, 0.4132407235172410], #row 1 no 1
                        [298.94, 295.8, 2.77, 4.0, 6.840000000000000, 2.45, 0.4692697997619050],#row 3 yes 2
                        [298.59, 296.63, 2.67, 6.0, 7.21, 2.58, 0.4692697997619050] #row 6 yes 2
                       ]
                       ,
                       columns=["ClimSST", "Temperature_Kelvin", "Temperature_Kelvin_Standard_Deviation", "SSTA_Frequency", "SSTA_Frequency_Standard_Deviation" ,"TSA_Frequency_Standard_Deviation" ,"mean_cur"])
test_r = pandas2ri.py2rpy(test_df)
pred = r.predict(model_R, test_r, type="prob", probability=True)

In [26]:
print(test_r)

  ClimSST Temperature_Kelvin Temperature_Kelvin_Standard_Deviation
0  297.28             295.60                                  2.15
1  298.94             295.80                                  2.77
2  298.59             296.63                                  2.67
  SSTA_Frequency SSTA_Frequency_Standard_Deviation
0             12                              3.34
1              4                              6.84
2              6                              7.21
  TSA_Frequency_Standard_Deviation  mean_cur
0                             1.10 0.4132407
1                             2.45 0.4692698
2                             2.58 0.4692698



In [27]:
print(pred)

          no       yes
0 0.48277556 0.5172244
1 0.01936671 0.9806333
2 0.01807990 0.9819201



In [28]:
print(pred[0])

[0.48277556 0.01936671 0.0180799 ]
